# Big Data Model and Evaluation

## Decision Tree

We construct a decition tree of depth 10 based on tokenized description and tags, and also other attributes like comment_disabled.

First import modules and initialize spark context

In [1]:
%load_ext autoreload
%autoreload 2

from predict.decision_tree import *
from pyspark import SparkContext
# setting this as the program was run in container
SparkContext.setSystemProperty('spark.driver.memory', '7g')
spark: SparkSession = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/11/20 07:00:15 WARN Utils: Your hostname, KW resolves to a loopback address: 127.0.1.1; using 172.24.210.113 instead (on interface eth0)
23/11/20 07:00:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 07:00:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Lazy-load the data into DataFrame. Pyspark DataFrame has similar API to pandas, and thus is more readable.

Then Filter and cast the data to correct types. Tokenize the description and tag fields for model later

In [2]:
file_list = [
    f"{ROOT_PATH}/{c}_youtube_trending_data_processed.csv" for c in COUNTRIES
]
df = spark.read.option("header", True).csv(file_list)
print(df)

df = filter_and_cast(df)
df = tokenize_description(df)
df = tokenize_tag(df)

DataFrame[video_id: string, title: string, publishedAt: string, channelId: string, channelTitle: string, categoryId: string, trending_date: string, tags: string, view_count: string, likes: string, dislikes: string, comment_count: string, thumbnail_link: string, comments_disabled: string, ratings_disabled: string, description: string, category_name: string]


[nltk_data] Downloading package punkt to /home/kw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Construct the feature hashing function and the decision tree model to be fitted

In [3]:
hashing_text, hashing_tags, assembler = construct_hashing_assembler()
df = hashing_text.transform(df)
df = hashing_tags.transform(df)
df = assembler.transform(df)

regressor = DecisionTreeRegressor(
    maxDepth=15,
    maxMemoryInMB=4096,
    featuresCol="features",
    labelCol="view_count",
    predictionCol="prediction",
)
model = regressor.fit(df)
print(model.featureImportances)

23/11/19 21:44:12 WARN MemoryStore: Not enough space to cache rdd_32_2 in memory! (computed 8.3 MiB so far)
23/11/19 21:44:12 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 8.3 MiB so far)
23/11/19 21:44:12 WARN MemoryStore: Not enough space to cache rdd_32_7 in memory! (computed 8.3 MiB so far)
23/11/19 21:44:12 WARN BlockManager: Persisting block rdd_32_7 to disk instead.
23/11/19 21:44:12 WARN BlockManager: Persisting block rdd_32_2 to disk instead.
23/11/19 21:44:12 WARN MemoryStore: Not enough space to cache rdd_32_1 in memory! (computed 8.3 MiB so far)
23/11/19 21:44:12 WARN BlockManager: Persisting block rdd_32_1 to disk instead.
23/11/19 21:44:12 WARN MemoryStore: Not enough space to cache rdd_32_4 in memory! (computed 8.3 MiB so far)
23/11/19 21:44:12 WARN BlockManager: Persisting block rdd_32_4 to disk instead.
23/11/19 21:44:12 WARN BlockManager: Persisting block rdd_32_0 to disk instead.
23/11/19 21:44:12 WARN MemoryStore: Not enough space to cach

(578,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,62,63,64,65,66,67,68,69,70,71,72,73,74,77,78,79,80,82,84,85,86,87,89,90,92,93,95,96,97,98,100,101,102,103,105,106,107,112,113,114,115,116,117,118,120,121,122,123,124,126,127,128,129,130,131,133,136,137,138,139,140,142,143,145,146,147,148,153,154,155,156,157,158,160,163,164,165,168,170,171,174,175,176,178,180,182,185,188,189,191,192,194,195,196,199,202,206,211,213,216,217,218,219,221,224,225,227,231,232,233,236,237,239,240,241,242,245,246,249,251,252,254,255,261,266,267,271,272,274,281,282,285,287,288,291,293,294,296,298,300,303,305,306,308,309,310,313,314,315,316,319,321,322,325,327,328,330,331,333,335,339,342,343,344,345,346,348,349,350,351,355,356,359,360,362,364,369,370,372,378,379,381,384,387,391,392,394,395,396,397,399,401,408,409,411,412,415,416,418,424,426,432,438,439,448,449,453,454,455,458,461,462,464

Different channels have vastly different number of subscribers. To better evaluate the model, we don't calculate the error on view count, but instead its rank inside the same channel.

In [4]:
file_list = [
    f"{TEST_PATH}/{c}_youtube_trending_data_processed_test.csv" for c in COUNTRIES
]
df = spark.read.option("header", True).csv(file_list[0])

df = filter_and_cast(df)
df = tokenize_description(df)
df = tokenize_tag(df)

df = hashing_text.transform(df)
df = hashing_tags.transform(df)
df = assembler.transform(df)
df = model.transform(df)

# Eval based on rank instead of abs view count
df = df.withColumn(
    "expected",
    dense_rank().over(Window.partitionBy("channelId").orderBy("view_count")),
)
df = df.withColumn(
    "predicted",
    dense_rank().over(Window.partitionBy("channelId").orderBy("prediction")),
)
df = df.withColumns(
    {
        "expected": df["expected"].cast("double"),
        "predicted": df["predicted"].cast("double"),
    }
)

metrics = RegressionMetrics(df.select("expected", "predicted").rdd.map(tuple))
print(f"MAE: {metrics.meanAbsoluteError}, MSE: {metrics.meanSquaredError}")

[nltk_data] Downloading package punkt to /home/kw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/kw/miniconda3/envs/CS5344/lib/python3.9/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


MAE: 11.692059653365579, MSE: 395.3808948004837


In [5]:
sub_df = df.limit(100).select(
    ["video_id", "title", "channelId", "channelTitle", "expected", "predicted"]
)
sub_df.write.csv(f"{PREDICT_OUTPUT}/decision_tree2", header=True)
print("Prediction sample saved!")

Prediction sample saved!


## Random Forest
Now we are adding some randomness into the trees and construct a Random Forest model

In [3]:
hashing_text, hashing_tags, assembler = construct_hashing_assembler()
df = hashing_text.transform(df)
df = hashing_tags.transform(df)
df = assembler.transform(df)

regressor = RandomForestRegressor(
    numTrees=10,
    maxDepth=12,
    maxMemoryInMB=4096,
    featuresCol="features",
    labelCol="view_count",
    predictionCol="prediction",
    subsamplingRate=0.5,
)

model = regressor.fit(df)

23/11/19 21:54:01 WARN MemoryStore: Not enough space to cache rdd_32_9 in memory! (computed 8.5 MiB so far)
23/11/19 21:54:01 WARN BlockManager: Persisting block rdd_32_9 to disk instead.
23/11/19 21:54:01 WARN MemoryStore: Not enough space to cache rdd_32_10 in memory! (computed 8.5 MiB so far)
23/11/19 21:54:01 WARN MemoryStore: Not enough space to cache rdd_32_1 in memory! (computed 8.5 MiB so far)
23/11/19 21:54:01 WARN BlockManager: Persisting block rdd_32_10 to disk instead.
23/11/19 21:54:01 WARN BlockManager: Persisting block rdd_32_1 to disk instead.
23/11/19 21:54:01 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 8.5 MiB so far)
23/11/19 21:54:01 WARN BlockManager: Persisting block rdd_32_0 to disk instead.
23/11/19 21:54:01 WARN MemoryStore: Not enough space to cache rdd_32_13 in memory! (computed 8.5 MiB so far)
23/11/19 21:54:01 WARN BlockManager: Persisting block rdd_32_13 to disk instead.
23/11/19 21:54:01 WARN MemoryStore: Not enough space to 

Evaluate the model using the same metric. We notice that performance is indeed better

In [4]:
print(model.featureImportances)
file_list = [
    f"{TEST_PATH}/{c}_youtube_trending_data_processed_test.csv" for c in COUNTRIES
]
df = spark.read.option("header", True).csv(file_list)

df = filter_and_cast(df)
df = tokenize_description(df)
df = tokenize_tag(df)

df = hashing_text.transform(df)
df = hashing_tags.transform(df)
df = assembler.transform(df)
df = model.transform(df)

# Eval based on rank instead of abs view count
df = df.withColumn(
    "expected",
    dense_rank().over(Window.partitionBy("channelId").orderBy("view_count")),
)
df = df.withColumn(
    "predicted",
    dense_rank().over(Window.partitionBy("channelId").orderBy("prediction")),
)
df = df.withColumns(
    {
        "expected": df["expected"].cast("double"),
        "predicted": df["predicted"].cast("double"),
    }
)

metrics = RegressionMetrics(df.select("expected", "predicted").rdd.map(tuple))
print(f"MAE: {metrics.meanAbsoluteError}, MSE: {metrics.meanSquaredError}")
sub_df = df.limit(100).select(
    [
        "video_id",
        "title",
        "channelId",
        "channelTitle",
        "expected",
        "predicted",
        "view_count",
        "prediction",
    ]
)
sub_df.write.csv(f"{PREDICT_OUTPUT}/random_forest", header=True)

(578,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,42,43,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,100,101,102,103,104,106,108,110,112,117,118,119,120,121,122,123,125,127,128,130,131,132,133,134,135,136,137,138,139,140,141,142,144,145,147,148,152,153,155,156,157,158,160,161,163,165,166,167,171,173,174,175,176,177,179,180,181,182,184,186,188,189,193,194,195,197,198,199,200,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,284,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,3

[nltk_data] Downloading package punkt to /home/kw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/kw/miniconda3/envs/CS5344/lib/python3.9/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


MAE: 9.714429665457477, MSE: 287.86638452237


## Gradient boosting
There's also possibility to combine week learners (trees) into a strong model. We explored Gradient Boosted Tree here

In [3]:
hashing_text, hashing_tags, assembler = construct_hashing_assembler()
df = hashing_text.transform(df)
df = hashing_tags.transform(df)
df = assembler.transform(df)

regressor = GBTRegressor(
    maxDepth=15,
    maxMemoryInMB=4096,
    featuresCol="features",
    labelCol="view_count",
    predictionCol="prediction",
    subsamplingRate=0.5
)

model = regressor.fit(df)

We noticed improvement in evaluation metric

In [4]:
print(model.featureImportances)
file_list = [
    f"{TEST_PATH}/{c}_youtube_trending_data_processed_test.csv" for c in COUNTRIES
]
df = spark.read.option("header", True).csv(file_list)

df = filter_and_cast(df)
df = tokenize_description(df)
df = tokenize_tag(df)

df = hashing_text.transform(df)
df = hashing_tags.transform(df)
df = assembler.transform(df)
df = model.transform(df)

# Eval based on rank instead of abs view count
df = df.withColumn(
    "expected",
    dense_rank().over(Window.partitionBy("channelId").orderBy("view_count")),
)
df = df.withColumn(
    "predicted",
    dense_rank().over(Window.partitionBy("channelId").orderBy("prediction")),
)
df = df.withColumns(
    {
        "expected": df["expected"].cast("double"),
        "predicted": df["predicted"].cast("double"),
    }
)

metrics = RegressionMetrics(df.select("expected", "predicted").rdd.map(tuple))
print(f"MAE: {metrics.meanAbsoluteError}, MSE: {metrics.meanSquaredError}")
sub_df = df.limit(100).select(
    [
        "video_id",
        "title",
        "channelId",
        "channelTitle",
        "expected",
        "predicted",
        "view_count",
        "prediction",
    ]
)
sub_df.write.csv(f"{PREDICT_OUTPUT}/GBT", header=True)

(578,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,60,61,62,63,64,65,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,125,126,127,128,129,130,132,133,134,135,137,139,141,142,143,144,145,146,147,148,149,151,152,153,154,155,156,157,159,160,161,162,163,164,165,166,167,169,170,172,173,174,175,176,177,178,179,180,183,184,185,186,187,188,190,191,193,195,196,197,198,200,201,202,203,205,207,209,210,211,213,214,215,217,218,219,220,221,222,223,224,225,226,227,228,230,231,232,233,234,235,238,239,240,241,242,243,244,245,246,247,249,250,251,252,254,255,256,257,258,259,260,261,262,263,264,265,266,268,269,270,271,272,273,274,275,276,279,280,281,282,286,287,288,291,292,293,294,295,296,297,298,299,300,301,302,304,305,306,307,309,310,311,312,31

[nltk_data] Downloading package punkt to /home/kw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
23/11/20 07:04:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
/home/kw/miniconda3/envs/CS5344/lib/python3.9/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


MAE: 8.979040709391375, MSE: 242.78516727126163
